In [ ]:
#G 6A

import hashlib  # For hashing the message using SHA-1
import random   # For generating random keys and nonces

# --- Parameters Setup (for demonstration purposes only) ---

# Small prime values for demo (not secure for real applications)
p = 23  # A small prime number
q = 11  # A prime number such that q divides (p-1), since 22 % 11 == 0

# Function to compute a suitable generator g
def calculate_g():
    for h in range(2, p-1):  # Try h from 2 to p-2
        g = pow(h, (p - 1) // q, p)  # g = h^((p-1)/q) mod p
        if g > 1:
            return g  # Return g if valid
    return None

g = calculate_g()  # Calculate generator

# --- Key Storage ---

private_key = None  # To store the private key x
public_key = None   # To store the public key y = g^x mod p

# --- Key Generation ---

def generate_keys():
    global private_key, public_key
    private_key = random.randint(1, q - 1)  # Select private key in [1, q-1]
    public_key = pow(g, private_key, p)    # Compute public key
    print("Keys generated!")
    print("Private key (x):", private_key)
    print("Public key (y):", public_key)

# --- Signing a Message ---

def sign_message(message):
    global private_key
    if private_key is None:
        print("Error: Private key not found. Please generate keys first.")
        return None, None

    # Hash the message using SHA-1
    h_obj = hashlib.sha1(message.encode('utf-8'))
    hash_int = int(h_obj.hexdigest(), 16)

    # Generate random k in [1, q-1]
    k = random.randint(1, q - 1)

    # Compute r = (g^k mod p) mod q
    r = pow(g, k, p) % q
    if r == 0:
        print("r = 0, choose a different k")
        return None, None

    # Compute the modular inverse of k mod q
    try:
        k_inv = pow(k, -1, q)
    except ValueError:
        print("No modular inverse for k; choose a different k")
        return None, None

    # Compute s = (k_inv * (hash + x * r)) mod q
    s = (k_inv * (hash_int + private_key * r)) % q
    if s == 0:
        print("s = 0, choose a different k")
        return None, None

    print("Message signed successfully!")
    print("Signature (r, s):", (r, s))
    return r, s

# --- Verifying a Signature ---

def verify_signature(message, r, s):
    if public_key is None:
        print("Error: Public key not found. Please generate keys first.")
        return False

    # Check that r and s are valid
    if not (0 < r < q and 0 < s < q):
        print("Invalid signature parameters.")
        return False

    # Hash the message
    h_obj = hashlib.sha1(message.encode('utf-8'))
    hash_int = int(h_obj.hexdigest(), 16)

    # Compute w = s^(-1) mod q
    try:
        w = pow(s, -1, q)
    except ValueError:
        print("Signature verification failed due to modular inverse error.")
        return False

    # Compute u1 = (hash * w) mod q and u2 = (r * w) mod q
    u1 = (hash_int * w) % q
    u2 = (r * w) % q

    # Compute v = ((g^u1 * y^u2) mod p) mod q
    v = (pow(g, u1, p) * pow(public_key, u2, p)) % p % q

    print("Computed v:", v)
    print("Provided r:", r)
    if v == r:
        print("Signature is valid!")
        return True
    else:
        print("Signature is invalid!")
        return False

# --- Menu for CLI interaction ---

def menu():
    while True:
        print("\n--- DSA Signature Scheme Menu ---")
        print("1. Generate Keys")
        print("2. Sign Message")
        print("3. Verify Signature")
        print("4. Exit")
        choice = input("Enter your choice: ").strip()

        if choice == '1':
            generate_keys()
        elif choice == '2':
            message = input("Enter the message to sign: ")
            sign_message(message)
        elif choice == '3':
            message = input("Enter the message to verify: ")
            try:
                r = int(input("Enter r (signature component): "))
                s = int(input("Enter s (signature component): "))
            except ValueError:
                print("Invalid input; please enter integer values for r and s.")
                continue
            verify_signature(message, r, s)
        elif choice == '4':
            print("Exiting.")
            break
        else:
            print("Invalid choice. Please try again.")

# --- Main Execution ---

if __name__ == "__main__":
    print("Digital Signature Standard (DSS) Example - DSA Implementation")
    print("Parameters (for demonstration):")
    print("p:", p, "q:", q, "g:", g)
    menu()


Digital Signature Standard (DSS) Example - DSA Implementation
Parameters (for demonstration):
p: 23 q: 11 g: 4

--- DSA Signature Scheme Menu ---
1. Generate Keys
2. Sign Message
3. Verify Signature
4. Exit
Enter your choice: 1
Keys generated!
Private key (x): 5
Public key (y): 12

--- DSA Signature Scheme Menu ---
1. Generate Keys
2. Sign Message
3. Verify Signature
4. Exit
Enter your choice: 2
Enter the message to sign: hello world
Message signed successfully!
Signature (r, s): (2, 4)

--- DSA Signature Scheme Menu ---
1. Generate Keys
2. Sign Message
3. Verify Signature
4. Exit
Enter your choice: 3
Enter the message to verify: hello world
Enter r (signature component): 2
Enter s (signature component): 3
Computed v: 4
Provided r: 2
Signature is invalid!

--- DSA Signature Scheme Menu ---
1. Generate Keys
2. Sign Message
3. Verify Signature
4. Exit
Enter your choice: 4
Exiting.


In [ ]:
#S 6

import hashlib  # For hashing the message using SHA-1
import random   # For generating random numbers

# Function to compute modular inverse using Extended Euclidean Algorithm
def modinv(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m               # Quotient
        m, a = a % m, m          # Remainder becomes new 'a'
        x0, x1 = x1 - q * x0, x0 # Update x0 and x1 for next iteration
    return x1 % m0               # Ensure result is positive

# Function to hash a message using SHA-1 and convert to integer
def hash_message(message):
    return int(hashlib.sha1(message.encode()).hexdigest(), 16)

# Step 1: Global parameters (for demo purposes)
p = 30803  # A large prime number
q = 491    # A prime divisor of p-1
g = 2      # Generator where g^q mod p == 1

# Step 2: Key generation
x = random.randint(1, q - 1)  # Private key: a random integer in range (1, q-1)
y = pow(g, x, p)              # Public key: y = g^x mod p

# Display the generated private and public keys
print("Private key (x):", x)
print("Public key (y):", y)

# Step 3: Signing a message
def sign(message):
    Hm = hash_message(message)  # Step 3.1: Hash the message into an integer

    while True:
        k = random.randint(1, q - 1)  # Step 3.2: Select a random nonce k in (1, q-1)
        r = pow(g, k, p) % q         # Step 3.3: Compute r = (g^k mod p) mod q
        if r == 0:
            continue                 # If r = 0, choose another k
        k_inv = modinv(k, q)         # Step 3.4: Compute modular inverse of k mod q
        s = (k_inv * (Hm + x * r)) % q  # Step 3.5: Compute s = k^-1 * (H(m) + x*r) mod q
        if s == 0:
            continue                 # If s = 0, choose another k
        break                        # Valid (r, s) pair found
    return (r, s)                    # Return the digital signature

# Step 4: Verifying a signature
def verify(message, r, s):
    # Step 4.1: Check if r and s are in the correct range
    if not (0 < r < q and 0 < s < q):
        return False

    Hm = hash_message(message)   # Step 4.2: Hash the message again
    w = modinv(s, q)             # Step 4.3: Compute modular inverse of s mod q
    u1 = (Hm * w) % q            # Step 4.4: u1 = H(m) * w mod q
    u2 = (r * w) % q             # Step 4.5: u2 = r * w mod q

    # Step 4.6: Compute v = ((g^u1 * y^u2) mod p) mod q
    v = ((pow(g, u1, p) * pow(y, u2, p)) % p) % q

    # Step 4.7: Signature is valid if v == r
    return v == r

# Demo: Signing and verifying a message
message = "Hello, this is a signed message!"  # The message to be signed
r, s = sign(message)  # Generate the signature
print("\nSignature:")
print("r =", r)
print("s =", s)

# Verifying the generated signature
valid = verify(message, r, s)
print("\nVerification:", " Valid Signature" if valid else " Invalid Signature")


In [ ]:
#G 6

# Install the cryptography package (used for cryptographic primitives)
# !pip install cryptography

# Import necessary modules from the cryptography library
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.utils import (
    encode_dss_signature, decode_dss_signature
)

# Function to generate a DSA private and public key pair
def generate_keys():
    # Generate a private key with 2048-bit key size
    private_key = dsa.generate_private_key(key_size=2048)
    # Extract the corresponding public key
    public_key = private_key.public_key()
    return private_key, public_key

# Function to sign a message using the private key
def sign_message(message, private_key):
    # Convert message to bytes
    message_bytes = message.encode()
    # Sign the message using SHA-256 hash
    signature = private_key.sign(message_bytes, hashes.SHA256())
    # Decode the DSS signature into (r, s) format
    r, s = decode_dss_signature(signature)
    return signature, r, s

# Function to verify the signature using the public key
def verify_signature(message, signature, public_key):
    # Convert message to bytes
    message_bytes = message.encode()
    try:
        # Try to verify the signature
        public_key.verify(signature, message_bytes, hashes.SHA256())
        return True  # Signature is valid
    except Exception:
        return False  # Signature is invalid

# Main menu function to interact with the user
def main():
    print("\n--- Digital Signature Standard (DSS) ---")
    private_key = None
    public_key = None
    signature = None
    message = ""

    while True:
        # Display menu options
        print("\nMenu:")
        print("1. Generate Keys")
        print("2. Sign Message")
        print("3. Verify Signature")
        print("4. Exit")
        ch = input("Enter your choice (1-4): ")

        if ch == "1":
            # Generate keys when option 1 is selected
            print("\nGenerating Public and Private Keys...")
            private_key, public_key = generate_keys()
            print("Keys generated successfully!")

        elif ch == "2":
            # Sign a message using the private key
            if not private_key:
                print("Please generate keys first.")
                continue
            message = input("Enter the message to sign: ")
            signature, r, s = sign_message(message, private_key)
            print("Signature created successfully!")
            print(f"Signature (r, s):\n  r = {r}\n  s = {s}")

        elif ch == "3":
            # Verify the signature of a signed message
            if not public_key or not signature:
                print("Please sign a message first.")
                continue
            print("Verifying the signature...")
            is_valid = verify_signature(message, signature, public_key)
            if is_valid:
                print("Signature is valid! Message is authentic.")
            else:
                print("Signature is invalid! Message may be tampered.")

        elif ch == "4":
            # Exit the program
            break
        else:
            # Handle invalid input
            print("Invalid choice! Please enter a number between 1 and 4.")

# Entry point of the script
if __name__ == "__main__":
    main()


In [2]:
"""
 What is DSS?
DSS (Digital Signature Standard) is a federal standard developed by the National Institute of Standards and Technology (NIST) for digital signatures.

It ensures authentication, data integrity, and non-repudiation of digital messages.

🧮 Cryptographic Basis
DSS uses the Digital Signature Algorithm (DSA), a variant of the ElGamal signature scheme.

Based on the mathematical problem of discrete logarithms, which is computationally hard to reverse without the private key.

🗝️ Key Components
Private Key (signing key):

Used to generate a digital signature on a message.

Must be kept secret by the signer.

Public Key (verification key):

Used by the recipient to verify the digital signature.

Can be shared openly.

Message Digest (Hash):

DSS uses a secure hash function (like SHA-256) to create a message digest.

Only the digest is signed, not the full message, which improves efficiency.

✍️ Signature Generation
The signature is created using the private key and the hash of the message.

The output is a pair of integers (r, s), which make up the digital signature.

These values are derived from a random number, the private key, and the message hash.

✅ Signature Verification
The recipient uses the public key to verify:

That the signature (r, s) corresponds to the hash of the original message.

That the message has not been tampered with.

If the signature is valid, it confirms:

The message was signed by the holder of the private key.

The message is authentic and intact.

🧾 Applications of DSS
Email authentication (e.g., signed emails).

Software code signing.

Legal document verification.

Secure communications in financial and governmental institutions.

🔒 Security Features
Data Integrity: Detects if message data has been altered.

Authentication: Confirms the identity of the signer.

Non-repudiation: Signer cannot deny signing the message.

Technical details, sizes, and cryptographic inputs/outputs:

---

## 🔧 **Technical Details of DSS / DSA**

### 🔑 **Key Sizes**
- **Private Key (x)**:
  - A random integer: `0 < x < q`
  - Secret key known only to the signer.
- **Public Key (y)**:
  - Computed as: `y = g^x mod p`
  - Publicly shared for verification.

- **Domain Parameters**:
  1. **p** – a large prime modulus (usually 1024–3072 bits).
  2. **q** – a 160–256 bit prime divisor of (p - 1).
  3. **g** – a generator of a subgroup of order `q` in `Z_p*`, calculated as:
     ```python
     g = h^((p-1)/q) mod p
     ```

### 🔐 **Cryptographic Sizes (Modern Standards)**
| Parameter | Size Range / Standard |
|----------|------------------------|
| `p` (modulus) | 1024, 2048, or 3072 bits |
| `q` (subgroup order) | 160 bits (for 1024-bit `p`) or 224/256 bits (for 2048/3072-bit `p`) |
| `x` (private key) | Same bit length as `q` |
| `y` (public key) | Same bit length as `p` |
| Hash function output | 160 to 256 bits (depending on algorithm, e.g., SHA-1, SHA-256) |
| Signature `(r, s)` | Each part is of size `q` bits |

---

### 🧾 **Message & Hashing**
- **Message**: Any input text string.
- **Hash Function**:
  - DSS originally used **SHA-1** (160 bits).
  - Modern implementations prefer **SHA-256** or stronger (256 bits).
- Only the **digest of the message** is signed.

---

### ✍️ **Signature Generation**
1. A **random ephemeral key** `k` is generated: `1 < k < q`
2. Signature components:
   - `r = (g^k mod p) mod q`
   - `s = (k^-1 * (HASH(message) + x*r)) mod q`
3. Signature: a pair of integers `(r, s)`, each of size ≈ `q` bits.

### ✅ **Signature Verification**
1. Compute:
   - `w = s^-1 mod q`
   - `u1 = (HASH(message) * w) mod q`
   - `u2 = (r * w) mod q`
2. Verify:
   - `v = ((g^u1 * y^u2) mod p) mod q`
   - Valid if `v == r`

---

## 📦 **In the Cryptography Library (`cryptography` module)**

- `dsa.generate_private_key(key_size=2048)`
  - Generates a private key with **2048-bit modulus** `p` and **256-bit** `q`.
  - Secure for modern standards.

- `sign(message_bytes, hashes.SHA256())`
  - Signs using the **DSA algorithm with SHA-256**, providing **256-bit digest**.
  - Signature format: DER-encoded, decoded using `decode_dss_signature()`.

- Signature output:
  - Raw signature: DER-encoded byte string.
  - Decoded into `(r, s)` integers.

---

### 🔍 Example Sizes Summary:
| Component        | Approx Size       | Notes |
|------------------|------------------|-------|
| `p`              | 2048 bits         | Modulus |
| `q`              | 256 bits          | Subgroup order |
| `x` (Private Key)| 256 bits          | Secret |
| `y` (Public Key) | 2048 bits         | Shared |
| `k`              | 256 bits          | Random per signature |
| `r`, `s`         | 256 bits each     | Signature components |
| Hash (SHA-256)   | 256 bits          | Digest used for signing |

---
"""


'\n What is DSS?\nDSS (Digital Signature Standard) is a federal standard developed by the National Institute of Standards and Technology (NIST) for digital signatures.\n\nIt ensures authentication, data integrity, and non-repudiation of digital messages.\n\n🧮 Cryptographic Basis\nDSS uses the Digital Signature Algorithm (DSA), a variant of the ElGamal signature scheme.\n\nBased on the mathematical problem of discrete logarithms, which is computationally hard to reverse without the private key.\n\n🗝️ Key Components\nPrivate Key (signing key):\n\nUsed to generate a digital signature on a message.\n\nMust be kept secret by the signer.\n\nPublic Key (verification key):\n\nUsed by the recipient to verify the digital signature.\n\nCan be shared openly.\n\nMessage Digest (Hash):\n\nDSS uses a secure hash function (like SHA-256) to create a message digest.\n\nOnly the digest is signed, not the full message, which improves efficiency.\n\n✍️ Signature Generation\nThe signature is created using t